# Projeto 3 - Ciência dos Dados

#### Stephanie Wertheimer, Thomas Pilnik e Victor Sznifer

Temos duas bases de dados: uma que fala sobre medições da quantidade de E. Coli nas praias de Chicago, e outra que mede diferentes características físicas da água. Nosso objetivo é buscar uma relação entre a proliferação de bactérias e os parâmetros utlizados para analisar a água. Faremos isso usando um modelo de regressão linear. Em seguida, faremos uma regressão logística que medirá a probabilidade de uma praia ser ou não própria para banho (fator regulado, por entre outros fatores, pela quantidade de E. Coli na água) de acordo com suas características físicas. 

In [ ]:
%matplotlib inline41
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import math

In [ ]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

In [ ]:
lab_data = pd.read_csv('Beach_Lab_Data.csv', sep=',')

In [ ]:
lab_data

In [ ]:

lab_data.columns = ["DNATestID", "Timestamp", "Beach", "DNA1","DNA2", "DNAMean", "CultureTestID","Sample1TimeStamp","Sample1Reading","Sample2Reading","ReadingMean","CultureNote","SampleInterval","Sample2TimeStamp","Latitude","Longitude","Location"]

In [ ]:
lab_data=pd.DataFrame(lab_data.loc[:, ["DNATestID", "Beach", "DNAMean","ReadingMean","Sample1TimeStamp"]])

lab_data

In [ ]:
water_quality = pd.read_csv('Beach_Water_Quality_-_Automated_Sensors.csv', sep=',')

water_quality.columns = ["Beach", "Timestamp", "WtrTemperature", "Turbidity","TransducerDepth", "WaveHeight", "WavePeriod", "BatteryLife","TimestampLabel","ID"]

In [ ]:
water_quality=pd.DataFrame(water_quality.loc[:, ["Beach", "WtrTemperature", "Turbidity","TransducerDepth", "WaveHeight", "WavePeriod", "TimestampLabel"]])


In [ ]:
water_quality

In [ ]:
df = lab_data.set_index('Beach').join(water_quality.set_index('Beach'), how='inner', lsuffix='_co2', rsuffix='_income')


In [ ]:
df